# Microsoft Presidio에서 PII 분석 프로세스 사용자 지정

이 노트북은 다음과 같은 다양한 사용자 정의 사용 사례를 다룹니다.

1. 새로운 유형의 PII 엔터티를 감지하도록 Presidio 조정

2. 새로운 언어로 PII 엔터티를 감지하도록 Presidio 조정

3. 새로운 유형의 감지 모듈을 Presidio에 포함하여 서비스 범위를 개선합니다.

In [1]:
from typing import List
import pprint

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult
from presidio_analyzer.recognizer_registry import RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngine, SpacyNlpEngine, NlpArtifacts
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

# 예1 : 거부 목록 기반 PII 인식

PII 처리 토큰 정의 / 이 경우 제목 목록이 됨

In [2]:
titles_list = ["Sir", "Ma'am", "Madam", "Mr.", "Mrs.", "Ms.", "Miss", "Dr.", "Professor"]

PatternRecognizer두 번째로 다음 을 전달하여 해당 제목을 스캔 하는 which를 생성해 보겠습니다 deny_list.

In [3]:
titles_recognizer = PatternRecognizer(supported_entity="TITLE", deny_list=titles_list)

인식기를 직접 호출

In [4]:
text1 = "I suspect Professor Plum, in the Dining Room, with the candlestick"
result = titles_recognizer.analyze(text1, entities=["TITLE"])
print(f"Result:\n {result}")

Result:
 [type: TITLE, start: 10, end: 19, score: 1.0]



마지막으로 이 새로운 인식기를 Presidio에서 사용하는 인식기 목록에 추가해 보겠습니다 AnalyzerEngine.

In [5]:
analyzer = AnalyzerEngine()
analyzer.registry.add_recognizer(titles_recognizer)

AnalyzerEnginePresidio는 NlpEngine엔티티를 감지하고 토큰, 보조 정리 및 기타 언어 기능을 추출하는 데 사용되는 것을 포함하여 사용 가능한 모든 인식기를 로드

새 인식기를 제자리에 놓고 분석기를 실행해 보겠습니다.

In [6]:
results = analyzer.analyze(text=text1, language="en")

In [7]:
print("Results:")
print(results)

Results:
[type: TITLE, start: 10, end: 19, score: 1.0, type: PERSON, start: 20, end: 24, score: 0.85]


"Plum"이라는 이름과 제목이 모두 PII로 식별

In [8]:
print("Identified these PII entities:")
for result in results:
    print(f"- {text1[result.start:result.end]} as {result.entity_type}")

Identified these PII entities:
- Professor as TITLE
- Plum as PERSON


# 예 2: 정규식 기반 PII 인식

추가할 수 있는 또 다른 간단한 인식기는 정규식을 기반으로 합니다.

극도로 보수적이고 숫자가 포함된 토큰을 PII로 취급한다고 가정해 보겠습니다.


In [9]:
# Define the regex pattern in a Presidio `Pattern` object:
numbers_pattern = Pattern(name="numbers_pattern",regex="\d+", score = 0.5)

# Define the recognizer with one or more patterns
number_recognizer = PatternRecognizer(supported_entity="NUMBER", patterns = [numbers_pattern])

In [10]:
text2 = "I live in 510 Broad st."

numbers_result = number_recognizer.analyze(text=text2, entities=["NUMBER"])
print("Result:")
print(numbers_result)

Result:
[type: NUMBER, start: 10, end: 13, score: 0.5]


 Presidio에 통합하기 전에 대표적인 데이터 세트에서 각 인식기를 테스트하는 것을 고려하십시오. 
 
 자세한 내용은 인식기 개발 모범 사례 문서를 참조하세요 .
 
 https://microsoft.github.io/presidio/analyzer/developing_recognizers/

# 예 3: 규칙 기반 논리 인식기

숫자 인식기를 한 단계 더 발전시켜 "Number One"과 같이 단어 내에서 숫자를 감지하고 싶다고 가정해 보겠습니다.

기본 spaCy 토큰 속성을 활용하거나 자체 논리를 작성하여 이러한 엔터티를 감지할 수 있습니다.

1. EntityRecognizer이 예제에서는 Presidio의 기본 인식기 를 구현하는 새 클래스를 만듭니다 . 이 추상 클래스를 사용하려면 load메서드와 analyze메서드를 구현해야 합니다.

2. 각 인식기는 NlpArtifacts입력 텍스트에 대해 미리 계산된 속성을 보유하는 type 의 개체를 허용합니다.

In [11]:
class MyRecognizer(EntityRecognizer):
    
    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(self, text: str, entities: List[str], nlp_artifacts: NlpArtifacts) -> List[RecognizerResult]:
        """
        Logic for detecting a specific PII
        """
        pass


In [12]:
class NumbersRecognizer(EntityRecognizer):
    
    expected_confidence_level = 0.7 # expected confidence level for this recognizer
    
    def load(self) -> None:
        """No loading is required."""
        pass

    def analyze(
        self, text: str, entities: List[str], nlp_artifacts: NlpArtifacts
    ) -> List[RecognizerResult]:
        """
        Analyzes test to find tokens which represent numbers (either 123 or One Two Three).
        """
        results = []
        
        # iterate over the spaCy tokens, and call `token.like_num`
        for token in nlp_artifacts.tokens:
            if token.like_num:
                result = RecognizerResult(
                    entity_type="NUMBER",
                    start=token.idx,
                    end=token.idx + len(token),
                    score=self.expected_confidence_level
                )
                results.append(result)
        return results


In [13]:
new_numbers_recognizer = NumbersRecognizer(supported_entities=["NUMBER"])

In [19]:
text3 = "Roberto lives in Five 10 Broad st."
analyzer = AnalyzerEngine()
analyzer.registry.add_recognizer(new_numbers_recognizer)

numbers_results2 = analyzer.analyze(text=text3, language="en")
print("Results:")
print("\n".join([str(res) for res in numbers_results2]))

Results:
type: PERSON, start: 0, end: 7, score: 0.85
type: NUMBER, start: 17, end: 21, score: 0.7
type: NUMBER, start: 22, end: 24, score: 0.7


# 예 5: 새로운 언어 지원

Presidio의 두 가지 주요 부분은 텍스트를 처리하며 새로운 언어가 필요한 경우 수정해야 합니다.

1. NlpEngine토큰화, 표제화, 명명된 엔티티 인식 및 기타 NLP 작업을 수행하는 NLP 모델을 포함합니다 .

2. 다른 PII 인식기( EntityRecognizer객체)를 조정하거나 생성해야 합니다.

# NLP 엔진 적응

내부 NLP 엔진인 Presidio는 spaCy와 Stanza를 모두 지원합니다. 

사용하기 전에 space/stanza에서 필요한 모델을 다운로드했는지 확인하십시오. 

자세한 내용은 여기를 참조하십시오 . 

예를 들어, 스페인어 중형 spaCy 모델을 다운로드하려면:python -m spacy download es_core_news_md

이 예에서는 영어와 스페인어로 된 NLP 모델과 함께 기본 NLP 프레임워크로 spaCy를 사용하도록 Presidio를 구성합니다.

In [22]:
!python -m spacy download es_core_news_md

2022-06-16 14:45:40.763029: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-16 14:45:40.763305: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



     ---------------------------------------- 42.3/42.3 MB 7.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [23]:
from presidio_analyzer.nlp_engine import NlpEngineProvider

#import spacy
#spacy.cli.download("es_core_news_md")

# Create configuration containing engine name and models
configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "es", "model_name": "es_core_news_md"},
               {"lang_code": "en", "model_name": "en_core_web_lg"}],
}

# Create NLP engine based on configuration
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine_with_spanish = provider.create_engine()

# Pass the created NLP engine and supported_languages to the AnalyzerEngine
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine_with_spanish, 
    supported_languages=["en", "es"]
)

# Analyze in different languages
results_spanish = analyzer.analyze(text="Mi nombre es Morris", language="es")
print("Results from Spanish request:")
print(results_spanish)

results_english = analyzer.analyze(text="My name is Morris", language="en")
print("Results from English request:")
print(results_english)

Results from Spanish request:
[]
Results from English request:
[type: PERSON, start: 11, end: 17, score: 0.85]


Presidio 지원 추가 NLP 모델 및 언어를 구성하는 방법에 대한 자세한 내용 은 이 문서 를 참조하십시오.

https://microsoft.github.io/presidio/analyzer/languages/

# 예 6: 문맥 단어 사용하기

Presidio에는 컨텍스트 단어를 활용하기 위한 내부 메커니즘이 있습니다.

이 메커니즘은 특정 단어가 그 앞이나 뒤에 나타나는 경우 PII 엔터티의 탐지 신뢰도를 높입니다.

이 예에서는 먼저 컨텍스트 없이 우편 번호 인식기를 구현한 다음 컨텍스트를 추가하여 신뢰도가 어떻게 변경되는지 확인합니다.

Zip 정규식 패턴(기본적으로 5자리)은 매우 주이므로 초기 신뢰도를 낮추고 컨텍스트 단어의 존재와 함께 증가하기를 원합니다.

In [24]:
# Define the regex pattern
regex = r"(\b\d{5}(?:\-\d{4})?\b)" # very weak regex pattern
zipcode_pattern = Pattern(name="zip code (weak)", regex=regex, score=0.01)

# Define the recognizer with the defined pattern
zipcode_recognizer = PatternRecognizer(supported_entity="US_ZIP_CODE", patterns = [zipcode_pattern])

registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer)
analyzer = AnalyzerEngine(registry=registry)

# Test
results = analyzer.analyze(text="My zip code is 90210",language="en")
print(f"Result:\n {results}")

Result:
 [type: US_ZIP_CODE, start: 15, end: 20, score: 0.01]


따라서 이것은 작동하지만 5자리 문자열을 잡을 것입니다. 

이것이 우리가 점수를 0.01로 설정한 이유입니다. 

점수를 높이기 위해 문맥 단어를 사용합시다:

In [25]:
# Define the recognizer with the defined pattern and context words
zipcode_recognizer = PatternRecognizer(supported_entity="US_ZIP_CODE", 
                                       patterns = [zipcode_pattern],
                                       context= ["zip","zipcode"])


생성할 때 매개변수 AnalyzerEngine에 전달하여 자체 컨텍스트 향상 논리를 제공할 수 있습니다 context_aware_enhancer.

AnalyzerEngine전달되지 않은 경우 기본적으로 생성 LemmaContextAwareEnhancer되며,

인식기가 컨텍스트 단어를 보유하고 해당 단어가 일치하는 엔터티의 컨텍스트에서 발견되는 경우 각 일치 결과의 점수가 향상됩니다.

In [26]:
registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer)
analyzer = AnalyzerEngine(registry=registry)

In [27]:
# Test
results = analyzer.analyze(text="My zip code is 90210",language="en")
print("Result:")
print(results)

Result:
[type: US_ZIP_CODE, start: 15, end: 20, score: 0.4]


이제 신뢰도 점수는 0.01이 아닌 0.4입니다.

기본 컨텍스트 유사성 계수는 ​​0.35이고 컨텍스트 유사성이 있는 기본 최소 점수는 0.4 이므로 및 매개변수를 값이 아닌 다른 값 LemmaContextAwareEnhancer으로 전달하여 변경할 수 있습니다.

예를 들면 다음과 같습니다.

context_similarity_factormin_score_with_context_similarityLemmaContextAwareEnhancer

In [28]:
registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer)
analyzer = AnalyzerEngine(
    registry=registry,
    context_aware_enhancer=
        LemmaContextAwareEnhancer(context_similarity_factor=0.45, min_score_with_context_similarity=0.4))

In [29]:
# Test
results = analyzer.analyze(text="My zip code is 90210",language="en")
print("Result:")
print(results)

Result:
[type: US_ZIP_CODE, start: 15, end: 20, score: 0.46]


신뢰도 점수는 0.01에서 0.45로 향상되었으며 최소값이 0.4 이상이므로 이제 0.46입니다.


Presidio는 분석기 수준에서 외부 컨텍스트 목록 전달을 지원합니다. 

이는 텍스트가 특정 열 또는 특정 사용자 입력 등에서 오는 경우에 유용합니다. 

"zip" 컨텍스트 단어가 텍스트에 나타나지 않지만 여전히 향상되는 방식에 주목하십시오. 

0.01에서 0.4까지의 신뢰 점수:

In [30]:
# Define the recognizer with the defined pattern and context words
zipcode_recognizer = PatternRecognizer(supported_entity="US_ZIP_CODE",
                                       patterns = [zipcode_pattern],
                                       context= ["zip","zipcode"])

registry = RecognizerRegistry()
registry.add_recognizer(zipcode_recognizer)
analyzer = AnalyzerEngine(registry=registry)

# Test
result = analyzer.analyze(text="My code is 90210",language="en", context=["zip"])
print("Result:")
print(result)

Result:
[type: US_ZIP_CODE, start: 11, end: 16, score: 0.4]


# 예 7: 결정 프로세스 추적

Presidio-analyzer의 결정 프로세스는 특정 PII가 감지된 이유에 대한 정보를 노출합니다. 이러한 정보에는 다음이 포함될 수 있습니다.

1. 엔터티를 감지한 인식기

2. 사용된 정규식 패턴

3. ML 모델의 해석 가능성 메커니즘

4. 어떤 문맥 단어가 점수를 향상시켰는지

5. 각 단계 전후의 자신감 점수 등.

자세한 내용은 의사결정 프로세스 문서 를 참조하십시오 .

https://microsoft.github.io/presidio/analyzer/decision_process/

우편 번호 값이 감지된 방법을 이해하기 위해 결정 프로세스 출력을 사용하겠습니다.

In [31]:
results = analyzer.analyze(text="My zip code is 90210",language="en", return_decision_process = True)
decision_process = results[0].analysis_explanation

pp = pprint.PrettyPrinter()
print("Decision process output:\n")
pp.pprint(decision_process.__dict__)

Decision process output:

{'original_score': 0.01,
 'pattern': '(\\b\\d{5}(?:\\-\\d{4})?\\b)',
 'pattern_name': 'zip code (weak)',
 'recognizer': 'PatternRecognizer',
 'score': 0.4,
 'score_context_improvement': 0.39,
 'supportive_context_word': 'zip',
 'textual_explanation': None,
 'validation_result': None}
